# Imports

In [1]:
from utils.nfl_data import (get_weekly_df, get_seasonal_df, get_roster_df,
                            create_mappings, create_position_groups, create_positional_df)

# Setting Years List

The years past into this list provides us with the data in each of those seasons.

In [2]:
years = [2023]

# Seasonal and Weekly Data

In [3]:
df_weekly = get_weekly_df(years=years, columns=[])

Downcasting floats.


In [4]:
df_seasonal = get_seasonal_df(years=years)

# Roster ID

For player mapping creations

In [5]:
columns = ["player_name", "player_id", "position"]
df_roster = get_roster_df(years=years, columns=columns)

# Create Player Mappings

In [6]:
player_map, id_map = create_mappings(df_roster=df_roster, df_seasonal=df_seasonal, df_weekly=df_weekly)

## Mapping Checks

In [7]:
# kelce_id = id_map["Travis Kelce"]

In [8]:
# player_map[kelce_id]["seasonal_df"]

In [9]:
# player_map[kelce_id]["week_dfs"][-1]

# Creating Position Groups

In [10]:
qbs, rbs, wrs, tes = create_position_groups(player_map=player_map)

In [11]:
df_qb = create_positional_df(qbs)
df_rb = create_positional_df(rbs)
df_wr = create_positional_df(wrs)
df_te = create_positional_df(tes)

In [12]:
df_qb.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,2,00-0026498,2023,REG,166,278,2070.0,8,7.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.163569


In [13]:
df_rb.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,16,00-0029892,2023,REG,0,0,0.0,0,0.0,0.0,...,0.037879,0.074067,0.029192,0.111111,0.048387,0.056338,0.070151,0.045576,0.276596,0.034116


In [14]:
df_wr.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,5,00-0027944,2023,REG,0,0,0.0,0,0.0,0.0,...,0.003846,0.096796,0.018395,0.166667,0.035714,0.058824,0.092531,0.048049,0.15942,0.04108


In [15]:
df_te.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,1,00-0024243,2023,REG,0,0,0.0,0,0.0,0.0,...,0.091286,0.051283,0.04829,0.0,0.037037,0.03125,0.024145,0.038632,0.375,0.020855


# Saving Data to CSV for Graphing Tools

In [16]:
def clean_df_for_csv(df):
    """"""
    df["name"] = df["player_id"].map(lambda x: player_map[x]['name'])
    df.drop(columns=["player_id"], inplace=True)
    
    df["total_epa"] = df["passing_epa"] + df["rushing_epa"]
    
    ignore_cols = ["Index", "name", "season", "season_type", "games", "target_share", "air_yards_share"
                  "tgt_sh", "ay_sh", "yac_sh", "ry_sh", "rtd_sh", "rfd_sh", "rtdfd_sh", "ppr_sh"]
    for col in df_qb.columns.to_list():
        if col not in ignore_cols:
            df[f"{col} per game"] = df[col] / df["games"]
    return df

## Quarterback Save

In [17]:
df_to_save_qb = clean_df_for_csv(df_qb)
df_to_save_qb.head(1)

,Index,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,wopr_x per game,special_teams_tds per game,fantasy_points per game,fantasy_points_ppr per game,tgt_sh per game,wopr_y per game,dom per game,w8dom per game,yptmpa per game,total_epa per game
0,2,2023,REG,166,278,2070.0,8,7.0,19.0,145.0,...,0.0,0.0,13.95,13.95,0.0,0.0,0.0,0.0,0.0,2.377215


In [18]:
df_to_save_qb.to_csv("FantasyData/advanced-stats/QB/2023.csv")

## Running Back Save

In [19]:
df_to_save_rb = clean_df_for_csv(df_rb)
df_to_save_rb.head(1)

,Index,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,wopr_x per game per game,special_teams_tds per game per game,fantasy_points per game per game,fantasy_points_ppr per game per game,tgt_sh per game per game,wopr_y per game per game,dom per game per game,w8dom per game per game,yptmpa per game per game,total_epa per game per game
0,16,2023,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.016775,0.0,0.42,0.66,0.001702,0.002963,0.002806,0.001823,0.011064,-0.076295


In [20]:
df_to_save_rb.to_csv("FantasyData/advanced-stats/RB/2023.csv")

# Wide Receiver Save

In [21]:
df_to_save_wr = clean_df_for_csv(df_wr)
df_to_save_wr.head(1)

,Index,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,wopr_x per game per game,special_teams_tds per game per game,fantasy_points per game per game,fantasy_points_ppr per game per game,tgt_sh per game per game,wopr_y per game per game,dom per game per game,w8dom per game per game,yptmpa per game per game,total_epa per game per game
0,5,2023,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.047834,0.0,1.775,2.275,0.01087,0.024199,0.023133,0.012012,0.039855,0.0


In [22]:
df_to_save_wr.to_csv("FantasyData/advanced-stats/WR/2023.csv")

# Tight End Save

In [23]:
df_to_save_te = clean_df_for_csv(df_te)
df_to_save_te.head(1)

,Index,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,wopr_x per game per game,special_teams_tds per game per game,fantasy_points per game per game,fantasy_points_ppr per game per game,tgt_sh per game per game,wopr_y per game per game,dom per game per game,w8dom per game per game,yptmpa per game per game,total_epa per game per game
0,1,2023,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.03899,0.0,0.6,1.1,0.007812,0.012821,0.006036,0.009658,0.09375,0.0


In [24]:
df_to_save_te.to_csv("FantasyData/advanced-stats/TE/2023.csv")